# Analyze results for 3D CGAN
Feb 22, 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


In [7]:
# img_size=64
img_size=128

In [8]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

### Read validation data

In [9]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=100

for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


## Read data

In [10]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [11]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [45]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202107*')]
n=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(n)

interactive(children=(Dropdown(description='x', options=('20210703_50233_cgan_128_nodes1_lr0.0002_vary_fm0_goo…

In [160]:
result=n.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210726_173009_cgan_128_nodes1_lr0.000002_finetune


## Plot Losses

In [161]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [162]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
6777,6777.0,28.0,0.479079,0.838290,1.317369,1.308797,inf,inf,-1.542795,inf,-2.601410,NaN,NaN,3.203735,0.060918,-0.958989,1.508852,0.00005,0.00005
6778,6778.0,28.0,0.490403,0.875251,1.365654,2.476866,inf,inf,-3.001800,inf,-1.710731,NaN,NaN,2.307626,-0.164366,-2.347204,1.509335,0.00005,0.00005
6779,6779.0,28.0,0.709167,0.436933,1.146101,1.723416,12.20095,10.477533,-1.922653,inf,-3.110786,NaN,NaN,0.280179,-2.138198,-1.517782,1.505827,0.00005,0.00005
6780,6780.0,28.0,0.498720,0.603727,1.102448,2.754047,inf,inf,-2.623415,inf,-2.232164,NaN,NaN,1.370276,-1.169111,-2.656648,1.497133,0.00005,0.00005
6781,6781.0,28.0,0.637469,0.403309,1.040778,1.398096,9.95773,8.559634,-1.326503,inf,-2.796693,NaN,NaN,-0.012716,-2.119793,-1.092377,1.493959,0.00005,0.00005
6782,6782.0,28.0,0.467696,0.453009,0.920705,2.284369,inf,inf,-2.377481,inf,-2.106086,NaN,NaN,1.520071,-1.000911,-2.138391,1.502611,0.00005,0.00005
6783,6783.0,28.0,0.570554,0.464865,1.035419,1.512811,inf,inf,-2.177765,inf,-2.668515,NaN,NaN,0.960536,-2.287899,-1.179450,1.504766,0.00005,0.00005
6784,6784.0,28.0,0.414550,0.580243,0.994793,1.987810,inf,inf,-2.988713,inf,-2.450733,NaN,NaN,2.221352,-0.323418,-1.799905,1.521746,0.00005,0.00005
6785,6785.0,28.0,0.436929,0.465771,0.902700,1.810077,inf,inf,-1.211430,inf,-3.125432,NaN,NaN,1.809919,-1.903997,-1.588386,1.501255,0.00005,0.00005
6786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,-1.700377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [164]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>30)].sort_values(by=['hist_chi']).head(10)

-3.003056287765503


/global/homes/v/vpa/.conda/envs/v3/lib/python3.8/site-packages/numpy/lib/function_base.py:3968: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g


In [165]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [166]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [167]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [168]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_spec2','chi_1a','chi_1b','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


In [169]:
df

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
0,0,10,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.015646,0.003533,0.797603,0.816782,0.000302,9.618322,...,59.745126,8,49,2,"[0.6921947428985842, 0.028067084114419952, 0.0...","[0.0003115955597243734, 0.0010052355143390832,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[296058067.9963533, 161814625.16915935, 883902...","[53829773.36651247, 26758027.018022466, 969743...",0-10
1,0,20,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.023243,0.005395,0.486629,0.515266,0.000343,8.198126,...,58.163529,8,36,1,"[0.6923616779102355, 0.02787156830414496, 0.01...","[0.0002942008127990135, 0.0009687725798798468,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[283014442.18089175, 154683253.61929524, 85958...","[50175237.864213504, 25296127.821582083, 88203...",0-20
2,0,30,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.022004,0.004140,0.830152,0.856296,0.000369,9.690053,...,58.125075,8,53,2,"[0.6925647869224949, 0.02693830147831542, 0.01...","[0.00030275593766711516, 0.0009748284499199238...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[280015437.20853716, 152016904.67307192, 83993...","[46720612.82131676, 25733548.86802462, 8865669...",0-30
3,0,40,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.010225,0.001251,2.840250,2.851725,0.000344,12.062545,...,60.442755,8,72,1,"[0.6920372323846816, 0.028569033513681712, 0.0...","[0.00031671420610483893, 0.0010058223344552582...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[301826260.2316023, 163500623.68427512, 906226...","[50559247.730280444, 26285505.309743773, 95393...",0-40
4,0,50,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.003138,0.001056,3.115652,3.119846,0.000370,14.714055,...,58.775193,8,91,6,"[0.6917157118950903, 0.02889547892811204, 0.02...","[0.00031262314841604946, 0.0009719503393310891...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[285841245.57302094, 155466462.96046832, 88303...","[46420768.18579187, 26211673.585640304, 952568...",0-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,28,6780,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.002123,0.001765,5.964496,5.968384,0.000602,17.247466,...,58.274374,8,138,6,"[0.6909278735802473, 0.030761575388596977, 0.0...","[0.00037606844185502313, 0.0011445435783578124...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[279327579.26798916, 156826219.60150042, 84721...","[44654113.55356691, 22090701.582847767, 107575...",28-6780
678,29,6790,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.002927,0.000985,2.984898,2.988811,0.000370,14.057265,...,59.701859,8,90,5,"[0.6918162197728028, 0.028205554514984237, 0.0...","[0.0003534234575204981, 0.0010801415919311098,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[263586674.66191173, 147188409.05497068, 79230...","[44739933.64137239, 20691435.873864666, 994309...",29-6790
679,29,6800,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.004121,0.000446,3.206598,3.211165,0.000344,13.695435,...,58.510525,8,95,6,"[0.6917339403373924, 0.028987620041088798, 0.0...","[0.00027896467374745223, 0.000834078743674943,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[272828315.80149776, 150471441.26487505, 79102...","[51443656.2164428, 18633360.144023843, 1024239...",29-6800
680,29,6810,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.003068,0.000353,2.962848,2.966269,0.000307,12.920473,...,58.931592,8,88,4,"[0.691228757390349, 0.0308654091096853, 0.0225...","[0.0002751606247131924, 0.0008050687282331199,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[264325006.50517133, 152033914.9085704, 816400...","[53310792.751487955, 18436328.877737828, 10110...",29-6810


### Slice best steps

In [170]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1a','chi_1b','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1a','chi_1b','chi_1c']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_spec2','chi_1a','chi_1b','chi_1c']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [171]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [172]:
df_sliced=w.result
[int(i.split('-')[1]) for i in df_sliced.label.values]

# df_sliced

[]

In [173]:
best_step=[]
df_test=df_merged.copy()
df_test=df_merged[(df_merged.epoch<5000)&(df_merged.epoch>0)]
cut_off=1.0

best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=8,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec2',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[1640 1650 1740 1950 2330 2580 3010 3020 5590 5780 5890 5940 5950 6090
 6380 6800]


array([1640, 1650, 1740, 1950, 2330, 2580, 3010, 3020, 5590, 5780, 5890,
       5940, 5950, 6090, 6380, 6800])

In [174]:
# best_step=[6176]
# best_step= [23985,24570,25155,25740,26325,26910,27495]
# best_step=[int(i.split('-')[1]) for i in df_sliced.label.values]
# best_step=np.arange(40130,40135).astype(int)

In [175]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(16, 64)
[(7, 1640), (7, 1650), (7, 1740), (8, 1950), (9, 2330), (11, 2580), (12, 3010), (12, 3020), (23, 5590), (24, 5780), (25, 5890), (25, 5940), (25, 5950), (26, 6090), (27, 6380), (29, 6800)]


In [176]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_spec2,sum_chi_1a,sum_chi_1b,sum_chi_1c
163,7-1640,7.497711,1.181852,0.452236,0.009550,0.010398,7.477763
164,7-1650,4.660035,1.077139,0.448659,0.008427,0.011930,4.639679
173,7-1740,9.388535,1.148954,0.440381,0.010827,0.029634,9.348074
194,8-1950,5.382815,1.134207,0.374116,0.005305,0.011793,5.365717
232,9-2330,3.149121,2.615752,1.087837,0.028310,0.002160,3.118652
257,11-2580,8.112328,1.145182,0.512790,0.007110,0.018191,8.087027
300,12-3010,3.750362,1.562736,0.179354,0.013885,0.020016,3.716461
301,12-3020,3.533120,1.829591,0.175638,0.017039,0.019060,3.497021
558,23-5590,5.791973,1.240465,0.666197,0.005639,0.015815,5.770519
577,24-5780,0.897965,1.437051,0.536028,0.030829,0.008287,0.858849


### Interactive plot

In [177]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type,img_size):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))
                y2=x2**2*y2; yerr2=x2**2*yerr2 ## Plot k^2 P(y)
                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:,0]
                print(images.shape)
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [178]:
interact_manual(f_plot_hist_spec,df=fixed(df_best),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=df_best.step.values),
                img_size=fixed(img_size),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type, img_size)>

In [159]:
best_step

array([1650, 1670, 1690, 1740, 1770, 1870, 1950, 2670, 2940, 3850, 4180,
       4230, 4680, 5150, 5690, 5890])

In [163]:
sigma=1.1
fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma)
print(fname)
a1=np.load(fname,mmap_mode='r')[-100:]
a1.shape

images=a1[:,0,:,:,0]
f_plot_grid(images[8:16],cols=4,fig_size=(8,4))


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy
2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [ ]:
# # fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
# fldr=result_dir
# print(fldr)
# flist=glob.glob(fldr+'/models/checkpoint_*.tar')
# len(flist)

In [ ]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [ ]:
# best_step

In [ ]:
# ! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy

In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.5_H70.0.npy'
# np.load(fname,mmap_mode='r').shape

In [65]:
2880/90

32.0